<a href="https://colab.research.google.com/github/Hj-lh/Fine-tune-qwen-for-PIC24/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-14B-Instruct",
    max_seq_length=2048,
    load_in_4bit= True,
    dtype=None
)

==((====))==  Unsloth 2024.12.12: Fast Qwen2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/210k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing= "unsloth",
    random_state= 3047,
    use_rslora= False,
    loftq_config= None,
)

Unsloth 2024.12.12 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [5]:
import pandas as pd
from datasets import Dataset
alpaca_prompt = """You are a PIC24HJ32GP202 microcontroller expert answering student instruction. Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["response"]
    texts = []

    for instruction, output in zip(instructions, outputs):
        # Format the prompt without the input field
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

from datasets import load_dataset

file_path = "data.csv"
dataframe = pd.read_csv(file_path)
dataset = Dataset.from_pandas(dataframe)

# dataset = load_dataset("data.csv", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)



Map:   0%|          | 0/634 [00:00<?, ? examples/s]

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/634 [00:00<?, ? examples/s]

In [7]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
10.326 GB of memory reserved.


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 634 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 68,812,800


Step,Training Loss
1,2.567700
2,2.126500
3,1.917500
4,2.244500
5,1.867200
6,1.728700
7,1.291500
8,1.355500
9,1.141800
10,0.919100


In [9]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "what is WDT ? and give a code example  ", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
decode = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decode[0])
#assume there is a system input to pin A0 with a wave with unknown frequency and the system will check if frequency bigger than 1kh or not if yes it will giv 1 as output else it will do nothing .output on Pin B3, measure the frequency input

You are a PIC24HJ32GP202 microcontroller expert answering student instruction. Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
what is WDT ? and give a code example  

### Response:

WDT stands for Watchdog Timer. It is used to reset the microcontroller if it gets stuck in an infinite loop. Example: `WDTCONbits.ON = 1;` enables the watchdog timer.



In [10]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "give a sample code of defining input and output on a specific pin and connect the output to LED and set a timer to toggle that ", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
decode = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decode[0])
#assume there is a system input to pin A0 with a wave with unknown frequency and the system will check if frequency bigger than 1kh or not if yes it will giv 1 as output else it will do nothing .output on Pin B3, measure the frequency input

You are a PIC24HJ32GP202 microcontroller expert answering student instruction. Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
give a sample code of defining input and output on a specific pin and connect the output to LED and set a timer to toggle that 

### Response:

```
#define _FOSCSEL_FCKSM_CSECMD & 0x0001
#define _FOSC_NOSC_PRI & 0x0006

#include <xc.h>
void main(void) {
    TRISBbits.TRISB0 = 0; // Output
    LATBbits.LATB0 = 0;   // Clear LED
    T2CON = 0x0000;
    PR2 = 50000;          // Period
    IEC0bits.T2IE = 1;    // Enable Timer2 interrupt
    _T2IF = 0;            // Clear Timer2 interrupt flag
    _T2IP = 1;            // Set Timer2 interrupt priority
    T2CONbits.TON = 1;    // Start Timer2
    while (1) {}
}
#pragma interrupt ISR
void ISR() {
    LATBbits.LATB0 = !_LATBbits.LATB0; // Toggle LED
    _T2IF = 0;                        // Clear Timer2 interrupt flag
}
```



In [11]:
model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 9.9G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.65 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


  0%|          | 0/48 [00:00<?, ?it/s]
We will save to Disk and not RAM now.
100%|██████████| 48/48 [03:51<00:00,  4.82s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00006.bin...
Unsloth: Saving model/pytorch_model-00002-of-00006.bin...
Unsloth: Saving model/pytorch_model-00003-of-00006.bin...
Unsloth: Saving model/pytorch_model-00004-of-00006.bin...
Unsloth: Saving model/pytorch_model-00005-of-00006.bin...
Unsloth: Saving model/pytorch_model-00006-of-00006.bin...
Done.
